In [1]:
import random
import string
from timeit import default_timer as timer

## 1. Zbiór danych wejściowych

In [2]:
input_data = ["bbb$", "aabbabd", "ababcd", "abaababaabaabaabab$"]
input_data

In [3]:
random_data = "".join(random.choice(string.ascii_letters + string.digits) for _ in range(100))
random_data

'V75mkzn41K74lwbCJzc3wPVqHknL9DzKOBaS0qrXlYZ8WmM3zWnG9TiqHkTgma4i5dLCcdW8078kkkww8w1BpNlm0sCs49wDwCI9'

In [4]:
with open("1997_714_head.txt", encoding="UTF-8") as f:
    text_data = f.read().replace("\n", " ")

In [5]:
text_data

'    Dz.U. z 1998 r. Nr 144, poz. 930                                                                                                                                                                                                     USTAWA                           z dnia 20 listopada 1998 r.                                                  o zryczałtowanym podatku dochodowym od niektórych przychodów                         osiąganych przez osoby fizyczne                                                                           Rozdział 1                                 Przepisy ogólne                                                                             Art. 1. Ustawa reguluje opodatkowanie zryczałtowanym podatkiem dochodowym niektórych przychodów (dochodów) osiąganych przez osoby fizyczne prowadzące pozarolniczą działalność gospodarczą oraz przez osoby duchowne.                                                                             Art. 2. 1. Osoby fizyczne osiągające prz

In [6]:
input_data.extend([random_data, text_data])

## 2. Upewnienie się, czy każdy łańcuch ma na końcu marker (znak dolara). Jak nie występuje na końcu to należy go dodać. W przypadku jak występuje na innych pozycjach niż koniec, to należy usunąć marker i zamienić dowolnym losowym znakiem (w tekście generowanym losowo) i dodać na końcu marker

In [7]:
def modify(data):
    marker = '$'
    
    if not data.endswith(marker):
        data += marker

    chars = string.ascii_letters + string.digits
    chars = chars.replace("$", "")
    
    list_data = list(data)
    for i in range(len(data)-1):
        if list_data[i] == marker:
            list_data[i] = random.choice(chars)
    
    data = "".join(list_data)
    return data

In [8]:
for i in range(len(input_data)):
    input_data[i] = modify(input_data[i])
    
input_data

['bbb$',
 'aabbabd$',
 'ababcd$',
 'abaababaabaabaabab$',
 'V75mkzn41K74lwbCJzc3wPVqHknL9DzKOBaS0qrXlYZ8WmM3zWnG9TiqHkTgma4i5dLCcdW8078kkkww8w1BpNlm0sCs49wDwCI9$',
 '    Dz.U. z 1998 r. Nr 144, poz. 930                                                                                                                                                                                                     USTAWA                           z dnia 20 listopada 1998 r.                                                  o zryczałtowanym podatku dochodowym od niektórych przychodów                         osiąganych przez osoby fizyczne                                                                           Rozdział 1                                 Przepisy ogólne                                                                             Art. 1. Ustawa reguluje opodatkowanie zryczałtowanym podatkiem dochodowym niektórych przychodów (dochodów) osiąganych przez osoby fizyczne prowadzące pozarolniczą dz

## 3. Algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu

In [9]:
class Trie:
    def __init__(self, value):
        self.children = {}
        self.value = value

    def build(self, text, i):
        if i != len(text):
            i_child = text[i]
            if i_child not in self.children:
                self.children[i_child] = Trie(i_child)
            i_next = i + 1
            self.children[i_child].build(text, i_next)

    def find(self, text, i):
        if i != len(text):
            if text[i] in self.children:
                i_next = i + 1
                i_child = text[i]
                return self.children[i_child].find(text, i_next)
            return False
        return True

In [10]:
def trie_match(text, pattern):
    n = len(text)
    val = None
    T = Trie(val)
    for i in range(n):
        T.build(text, i)
    
    start_i = 0
    result = T.find(pattern, start_i)
    return result

## 5. Porównanie wyników uzyskanych za pomocą struktury trie z algorytmem wyszukiwania wzorców KMP

In [11]:
def get_longest_prefix(str_):
    longest = [0 for _ in range(len(str_))]
    j = 0
    for q in range(1, len(str_)):
        while j > 0 and str_[j] != str_[q]:
            j = longest[j-1]
        if str_[j] == str_[q]:
            j += 1
        longest[q] = j
    return longest

def kmp_match(text, pattern, longest_prefix):
    n = len(text)
    m = len(pattern)
    i = 0
    j = 0 
    result = [] 

    while i < n:
        if text[i] == pattern[j]:
            i += 1
            j += 1
            if j == m:
                result.append(i-m)
                j = longest_prefix[j-1]
        else:
            if j > 0:
                j = longest_prefix[j-1]
            else:
                i += 1

    if result:
        return True
    else:
        return False

Porównanie wyszukiwania za pomocą KMP i struktury Trie:

- test 1

In [12]:
pattern = "bb"

In [13]:
longest_prefix = get_longest_prefix(input_data[0])
kmp_match(input_data[0], pattern, longest_prefix)

True

In [14]:
trie_match(input_data[0], pattern)

True

- test 2

In [15]:
pattern = "b."

In [16]:
longest_prefix = get_longest_prefix(input_data[0])
kmp_match(input_data[0], pattern, longest_prefix)

False

In [17]:
trie_match(input_data[0], pattern)

False

- test 3

In [18]:
pattern = "bab"

In [19]:
longest_prefix = get_longest_prefix(input_data[1])
kmp_match(input_data[1], pattern, longest_prefix)

True

In [20]:
trie_match(input_data[1], pattern)

True

- test 4

In [21]:
pattern = "abaababaabaabaabab"

In [22]:
longest_prefix = get_longest_prefix(input_data[3])
kmp_match(input_data[3], pattern, longest_prefix)

True

In [23]:
trie_match(input_data[3], pattern)

True

- test 5

In [24]:
pattern = " "

In [25]:
longest_prefix = get_longest_prefix(input_data[3])
kmp_match(input_data[3], pattern, longest_prefix)

False

In [26]:
trie_match(input_data[3], pattern)

False

- test 6

In [27]:
pattern = "Nr 139,poz. 932-934"

In [28]:
longest_prefix = get_longest_prefix(input_data[-1])
kmp_match(input_data[-1], pattern, longest_prefix)

False

In [29]:
trie_match(input_data[-1], pattern)

False

## 6. Testy czasowe

In [30]:
def test_kmp_match(text, pattern):
    start = timer()
    longest_prefix = get_longest_prefix(pattern)
    kmp_match(text, pattern, longest_prefix)
    end = timer()
    
    return end-start

In [31]:
def test_trie_match(text, pattern):
    start = timer()
    trie_match(text, pattern)
    end = timer()
    
    return end-start

In [32]:
def time_tests():
    tests = ["bb", 
             "abb", 
             "ababcd", 
             "ababaa", 
             input_data[4][10:40],
             "Przychodów (dochodów) opodatkowanych w formach zryczałtowanych nie łączy się zprzychodami (dochodami) z innych źródeł"]
    
    for i in range(len(input_data)):
        t_kmp = test_kmp_match(input_data[i], tests[i])
        t_trie = test_trie_match(input_data[i], tests[i])
        print("-"*40)
        print(f"Test dla danych wejściowych nr {i+1}")
        print(f"Algorytm KMP: {t_kmp} [s]")
        print(f"Trie: {t_trie} [s]")

In [33]:
time_tests()

----------------------------------------
Test dla danych wejściowych nr 1
Algorytm KMP: 6.100000000230921e-06 [s]
Trie: 1.330000000088205e-05 [s]
----------------------------------------
Test dla danych wejściowych nr 2
Algorytm KMP: 6.899999998921658e-06 [s]
Trie: 3.200000000092018e-05 [s]
----------------------------------------
Test dla danych wejściowych nr 3
Algorytm KMP: 6.000000000838668e-06 [s]
Trie: 2.360000000045659e-05 [s]
----------------------------------------
Test dla danych wejściowych nr 4
Algorytm KMP: 8.800000001585317e-06 [s]
Trie: 0.00016650000000062448 [s]
----------------------------------------
Test dla danych wejściowych nr 5
Algorytm KMP: 2.86000000002673e-05 [s]
Trie: 0.0044621999999989725 [s]
----------------------------------------
Test dla danych wejściowych nr 6
Algorytm KMP: 0.0005379999999988172 [s]
Trie: 10.461276 [s]


## 7. Wnioski

W testach czasowych widoczna jest znacząca różnica pomiędzy algorytmem bazującym na Trie oraz KMP.
Pierwszy z nich jest nawet o kilka rzędów wielkości wolniejszy od KMP ze względu na to, że budowa struktury Trie dla dużych tekstów jest bardzo kosztowna obliczeniowo i pamięciowo. Spowodowane jest to między innymi tym, że konstrukcja struktury Trie wymaga iteracji po każdym znaku w tekście i dodawania odpowiednich węzłów.